In [ ]:
%matplotlib inline
import matplotlib as mpl
import numpy as np

import os, sys
import glob
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, interact_manual
from ipywidgets import RadioButtons, fixed 

import seaborn as sns
sns.set(context="poster", font_scale=1.5, style="ticks")


## import from local files

## Boilerplate path hack to give access to full clustered_SNe package
import sys, os
if __package__ is None:
    if os.pardir not in sys.path[0]:
        file_dir = os.getcwd()
        sys.path.insert(0, os.path.join(file_dir, 
                                        os.pardir, 
                                        os.pardir))

from clustered_SNe import data_dir_default
from clustered_SNe.analysis.constants import m_proton, pc, yr, M_solar, \
                                   gamma, E_0, metallicity_solar
from clustered_SNe.analysis.sedov.dimensionalize_sedov import dimensionalized_sedov
from clustered_SNe.analysis.sedov.closed_form_sedov import SedovSolution
from clustered_SNe.analysis.parse import RunSummary, Overview, cols
from clustered_SNe.analysis.visualize_helpers import single_run, \
                                                     conduction_comparisons, \
                                                     SNe_distplot, \
                                                     plot_zones, \
                                                     plot_shock_location, \
                                                     plot_energy, \
                                                     plot_momentum, \
                                                     plot_luminosity

        
# Holds the most recent run, if more analysis is desired
run_summary = RunSummary()    

In [ ]:
mpl.rcParams['savefig.dpi'] = 80*2
mpl.rcParams['figure.dpi'] = 80*2
mpl.rcParams['figure.figsize'] = np.array((10,6))*1.5
mpl.rcParams['figure.facecolor'] = "white"

In [ ]:
from IPython.core.display import display, HTML


if widgets.version_info[0]==5:
    display(HTML("""
    <style>
    .widget-hbox .widget-label { min-width: 225px; }
    .widget-text { width: 600px; }
    .widget-hslider { width: 600px; }
    .widget-checkbox { width: 480px; }
    </style>"""))

In [ ]:
data_dir = "../src/turbulent-conduction-11-1e0-hires"
# data_dir = data_dir_default

# reference
# ../src/turbulent_11-1e0-better_cooling_units
# no cooling
# ../src/turbulent_11-1e0-adiabatic
# no turb 
# ../src/turbulent_11-1e0-no_turb
# no amr
# ../src/turbulent_11-1e0-better_cooling_units-no_amr

# you don't need the full uuid, just enough to distinguish it
id_default = ""

result_widget = interact_manual(single_run,
                                data_dir = widgets.Text(data_dir),
                                id = widgets.Text(id_default))
def update_run_summary_single_run(widget):
    """Copy the latest result into run_summary
    
    Don't reuse this function definition; needs to be bound to the correct result_widget"""
    run_summary.replace_with(result_widget.widget.result)
result_widget.widget.children[0].on_submit(update_run_summary_single_run)
result_widget.widget.children[1].on_submit(update_run_summary_single_run)
result_widget.widget.children[2].on_click( update_run_summary_single_run)



In [ ]:
run_summary.times[456:461] / (yr * 1e6)

In [ ]:
run_summary.E_tot[456:461]

In [ ]:
run_summary.times[-10:] / (1e6*yr)

In [ ]:
@interact(k=(0,run_summary.times.size-1,1))
def show(k):
    i_low = np.searchsorted( run_summary.df.loc[k].Radius, run_summary.R_shock[k]/pc*(1 - .25))[0]
    i_high = np.searchsorted( run_summary.df.loc[k].Radius, run_summary.R_shock[k]/pc*(1 + .25))[0]


    plt.plot( run_summary.df.loc[k].Radius.loc[i_low:i_high], run_summary.df.loc[k].Pressure.loc[i_low:i_high] )


    plt.yscale("log")
    plt.xlabel("Radius [pc]")
    plt.ylabel("Pressure [cgs units]")


In [ ]:
print(run_summary.first_unreasonable_energy())
print(run_summary.is_time_resolved())
print(run_summary.is_converged())

In [ ]:
plot_zones(run_summary)

In [ ]:
plot_shock_location(run_summary)

In [ ]:
Myr = 1e6*yr
km = 1000 * 100
s = 1

c_s = run_summary.df.iloc[-1].C_ad

v_shock = np.diff(run_summary.R_shock) / np.diff(run_summary.times)
v_shock[v_shock<1] = 1
plt.plot(run_summary.times[1:] / Myr, v_shock / (km /s ),
         label="simulation")

plt.axhline(c_s / (km / s), 
            linestyle="dotted", color="red",
            label="$c_s$ (final)")

plt.axvline(run_summary.times[run_summary.momentum.argmax()] / Myr,
            linestyle="dashed", color="black",
            label="Time of Max Momentum")

plt.legend(loc="best")

plt.yscale("log")

plt.xlabel("t [Myr]")
plt.ylabel("$v_\mathrm{shock}$ [km s$^{-1}$]")

plt.ylim(bottom=.1)

sns.rugplot(run_summary.overview.SNe_times / Myr, color="black", linewidth=3)

In [ ]:
_ = interact(plot_energy,
         run_summary=fixed(run_summary),
         x_axis = RadioButtons(options=["time", "checkpoints"]))

In [ ]:
def plot_energy_tmp(run_summary, x_axis):
    plt.figure()
    
    if x_axis == "time":
        x_variable = run_summary.times / yr
        xlabel = "Time [yr]"
        xscale = "linear"
        plt.xscale(xscale)
        xfmt = plt.gca().get_xaxis().get_major_formatter() # needs to be set AFTER plt.xscale()
        if xscale == "log":
            mask = x_variable > 1
        elif xscale == "linear":
            mask = np.full_like(x_variable, True, dtype=bool) 
            xfmt.set_powerlimits((-2, 2)) # force scientific notation outside this range

    elif x_axis == "checkpoints":
        x_variable = np.arange(len(run_summary.times))
        xlabel = "Checkpoint"
        xscale = "linear"
        mask = np.full_like(x_variable, True, dtype=bool) 

        plt.xscale(xscale)
        xfmt = plt.gca().get_xaxis().get_major_formatter() # needs to be set AFTER plt.xscale()

    else:
        raise NotImplementedError("can't recognize x_axis value: " + x_axis)


    plt.plot(x_variable[mask], run_summary.E_tot[mask], label="E_tot" )
#     plt.plot(x_variable[mask], run_summary.E_kin[mask], label="E_kin" )
#     plt.plot(x_variable[mask], run_summary.E_int[mask], label="E_int" )
    plt.legend(loc="best")
    plt.xscale(xscale)
    plt.xlabel(xlabel) 
    plt.gca().xaxis.set_major_formatter(xfmt)
    plt.ylabel("Energy [erg]")
    SNe_distplot(run_summary, x_axis)


    if x_axis == "checkpoints":
        plt.xlim(left=0)
        
plot_energy_tmp(run_summary, "time")
plt.xlim(
    left=4e6,
    right=4.2e6)

plt.axvline(1.28146e+14 / yr, linestyle="dashed", color="black")
plt.ylim(top=3.6e51, bottom=2.9e51)
# plt.ylim(top=3.305e51, bottom=3.304e51)



In [ ]:
plot_energy_tmp(run_summary, "time")
plt.xlim(
#     left=4e6,
    right=5e6,
)

# plt.axvline(1.28146e+14 / yr, linestyle="dashed", color="black")
# plt.ylim(top=3.6e51, bottom=2.9e51)
# plt.ylim(top=3.305e51, bottom=3.304e51)


In [ ]:
_ = interact(plot_momentum,
         run_summary=fixed(run_summary),
         x_axis = RadioButtons(options=["time", "checkpoints"]),
         clear_previous=fixed(True),
         distplot=fixed(True),
         y_axis_scaling=fixed("SNe"),
         plot_times_in_Myr=fixed(True),
         with_momentum_axvline=fixed(True),
         label=fixed(""))

In [ ]:
_ = interact(plot_luminosity,
         run_summary=fixed(run_summary),
         x_axis = RadioButtons(options=["time", "checkpoints"]))
